In [ ]:
import os
import sys

if sys.platform in ["win32"]:
    os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# Create a cache directory to store pretrained model.
cache_dir = os.path.join(".", "cache_models")
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

In [ ]:
from onnxruntime.transformers.models.gpt2.gpt2_helper import Gpt2Helper, MyGPT2LMHeadModel
from transformers import AutoConfig
import torch

model_name_or_path = "custom_q_and_a"
config = AutoConfig.from_pretrained(model_name_or_path, cache_dir=cache_dir)
model = MyGPT2LMHeadModel.from_pretrained(model_name_or_path, config=config, cache_dir=cache_dir)
device = torch.device("cpu")
model.eval().to(device)

print(model.config)

num_attention_heads = model.config.n_head
hidden_size = model.config.n_embd
num_layer = model.config.n_layer

In [ ]:
onnx_model_path = "gpt2.onnx"

In [ ]:
!{sys.executable} -m onnxruntime.transformers.models.gpt2.convert_to_onnx -m $model_name_or_path --output $onnx_model_path -o -p fp32 -t 10 >export_output.txt 2>&1

In [ ]:
file = open("export_output.txt", "r")
for line in file.readlines():
    if "Optimized operators" in line:
        print(line)